# Data Preparation

O objetivo aqui é preparar os dados para o treinamento criando um pipeline fitado para o conjunto de treinamento que seja usado no conjunto de teste evitando vazamento de informaçoes de teste para o treinamento

#Imports

In [0]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import joblib

#Funçoes

In [0]:
# Função que deixa colunas minúsculas
def normalize_columns(df):
    df.columns = [unidecode(col).lower() for col in df.columns]
    return df

# Função para normalizar os valores das colunas de texto
def normalize_text_values(df):
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].apply(lambda x: unidecode(x).lower() if isinstance(x, str) else x)
    return df

In [0]:
def percent_nulls(df):
    return round(df.isnull().mean() * 100, 2)

def unique_values(df):
    return df.nunique()

def percent_zeros(df):
    return round((df == 0).mean() * 100, 2)

def create_metadata(x_train):
    '''
    Criando metadados do conjunto de treino engenhado
    '''
    metadata = pd.DataFrame({
        'variavel': x_train.columns,
        'tipo': x_train.dtypes.astype(str),
        'percentual_nulos': percent_nulls(x_train),
        'valores_unicos': unique_values(x_train),
        'percentual_zeros': percent_zeros(x_train)
    })

    return metadata.reset_index(drop=True, inplace=True)

#Lendo dados de treino

# Data Prep do x_enginneered

In [0]:
categ_cols = [
    'sexo',
    'signo',
    'fumante_x_regiao',
    'fumante_x_classe',
    'facebook_x_regiao',
    'facebook_x_classe',
    'fumante_x_facebook',
    'cat_fumante',
    'cat_facebook',
    'cat_idade',
    'cat_imc',
    'cat_filhos',
    'cat_regiao',
    'cat_classe'
]

In [0]:
df = pd.read_excel('data/Seguro Saúde - Modelagem.xlsx', sheet_name='MODELAGEM')

x_train = pd.read_csv('data/x_enginneered.csv', na_values=["nan", "NaN", "None", ""])

for col in categ_cols:
    x_train[col] = x_train[col].astype('object')

df = normalize_columns(df)
df = normalize_text_values(df)

y_train = df[['matricula','valor']]

display(x_train.head())
display(y_train.head())

In [0]:
metadata = create_metadata(x_train)

display(metadata)

In [0]:
# Variáveis com base no metadata
numericas = metadata.query("tipo == 'float64'")["variavel"].tolist()

# Categóricas binárias: tipo object/category e apenas 2 valores únicos
binarias = metadata.query("(tipo == 'object') and valores_unicos == 2")["variavel"].tolist()

# Categóricas (mais de duas categorias)
categoricas = metadata.query("(tipo == 'object') and valores_unicos > 2")["variavel"].tolist()

print(f'Numéricas:\n{numericas}')
print(f'Binárias:\n{binarias}')
print(f'Categóricas:\n{categoricas}')

In [0]:
# Numéricas: mediana + scaler
num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Categóricas binárias: most frequent + one-hot (drop first)
bin_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

# Categóricas com múltiplos valores: most frequent + one-hot
cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

# Monta o pipeline completo com o ColumnTransformer
full_pipeline = ColumnTransformer(transformers=[
    ("num", num_pipeline, numericas),
    ("bin", bin_pipeline, binarias),
    ("cat", cat_pipeline, categoricas)
])

In [0]:
x_enginneered_prepared = full_pipeline.fit_transform(x_train.drop(columns=["matricula"]))

In [0]:
feat_names = full_pipeline.get_feature_names_out()

x_prepared_df = pd.DataFrame(x_enginneered_prepared, columns=feat_names, index=x_train.index)
x_prepared_df = pd.concat([x_train['matricula'], x_prepared_df], axis=1)

display(x_prepared_df.head())

In [0]:
print(x_prepared_df.shape)

In [0]:
joblib.dump(full_pipeline, 'artifacts/pipeline_engineered_features.pkl')

In [0]:
x_prepared_df.to_parquet('data/x_prepared.parquet')